In [1]:
import numpy as np
import os
import cv2
import json
import pandas as pd
from nerfstudio.data.utils.colmap_parsing_utils import *

# Functions

In [2]:
def compute_laplacian(images_paths):
    varis = []
    for k, image_path in enumerate(os.listdir(images_paths)):
        # if image_path.endswith('.jpg'):
        if image_path.endswith('.png'):
            image = cv2.imread(os.path.join(images_paths, image_path))
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            var = cv2.Laplacian(gray, cv2.CV_64F).var()
            varis.append(var)
            if k % 50 == 0:
                print_progress_bar(k, len(os.listdir(images_paths)))
    varis = np.array(varis)
    return varis

def print_progress_bar(iteration, total, bar_length=50):
    progress = (iteration / total)
    arrow = '█'
    spaces = ' ' * (bar_length - int(progress * bar_length))
    print(f'\rProgress: [{arrow * int(progress * bar_length)}{spaces}] {progress * 100:.2f}%', end='', flush=True)

def extract_time(time_file, content=None, nerfacto=True, splatfacto=True):
    with open(time_file) as file:
        df = pd.read_csv(file, sep='\s+')
    files = list(df[df['1'] == 'COLMAP']['2'])
    times = [*map(float, list(df[df['1'] == 'time:']['2']))]
    iterations = [*map(int, list(df[df['1'] == 'colmap_iterations:']['2']))]
    content = {}
    for k in range(len(files)):
        content[files[k]] = {}
        content[files[k]]['colmap'] = {}
        content[files[k]]['colmap']['time'] = times[k]
        content[files[k]]['colmap']['repetitions'] = iterations[k]
    if nerfacto:    
        files = list(df[df['1'] == 'nerfacto']['2'])
        for k in range(len(files), len(files) + len(files)):
            content[files[k-len(files)]]['nerfacto'] = {}
            content[files[k-len(files)]]['nerfacto']['time'] = times[k]
        if splatfacto:
            files = list(df[df['1'] == 'splatfacto']['2'])
            for k in range(len(files) + len(files), len(files) + len(files) + len(files)):
                content[files[k-len(files) - len(files)]]['splatfacto'] = {}
                content[files[k-len(files) - len(files)]]['splatfacto']['time'] = times[k]
    else:
        if splatfacto:
            files = list(df[df['1'] == 'splatfacto']['2'])
        for k in range(len(files), len(files) + len(files)):
            content[files[k-len(files)]]['splatfacto'] = {}
            content[files[k-len(files)]]['splatfacto']['time'] = times[k]
            
    return content

def count_number_images(path, content):
    for key in content.keys():
        diri = os.path.join(path, key, 'images')
        content[key]['number_images'] = len(os.listdir(diri))
        diri_colmap = os.path.join(path, key, 'colmap/sparse')
        if len(os.listdir(diri_colmap)) > 1 or len(os.listdir(diri_colmap)) == 0:
            content[key]['errado'] = True
        else:
            content[key]['errado'] = False

def add_evaluations(path, content, nerfacto=True, splatfacto=True):
    elems = [*range(10000, 100000, 10000)]
    elems.append(99999)
    for key in content.keys():
        if nerfacto:
            # Nerfacto
            if content[key]['nerfacto'].get('metrics') is None:
                content[key]['nerfacto']['metrics'] = {}
            if os.path.exists(os.path.join(path, key)):
                for elem in elems:
                    if os.path.exists(os.path.join(path, key, 'output_'+ str(elem) + '.json')):
                        with open(os.path.join(path, key, 'output_'+ str(elem) + '.json')) as file:
                            data = json.load(file)
                        if content[key]['nerfacto']['metrics'].get(elem) is None:
                            content[key]['nerfacto']['metrics'][elem] = {}
                        content[key]['nerfacto']['metrics'][elem]['psnr'] = data['results']['psnr']
                        content[key]['nerfacto']['metrics'][elem]['psnr_std'] = data['results']['psnr_std']
                        content[key]['nerfacto']['metrics'][elem]['ssim'] = data['results']['ssim']
                        content[key]['nerfacto']['metrics'][elem]['ssim_std'] = data['results']['ssim_std']
                        content[key]['nerfacto']['metrics'][elem]['lpips'] = data['results']['lpips']
                        content[key]['nerfacto']['metrics'][elem]['lpips_std'] = data['results']['lpips_std']
                        content[key]['nerfacto']['metrics'][elem]['num_rays_per_sec'] = data['results']['num_rays_per_sec']
                        content[key]['nerfacto']['metrics'][elem]['num_rays_per_sec_std'] = data['results']['num_rays_per_sec_std']
                        content[key]['nerfacto']['metrics'][elem]['fps'] = data['results']['fps']
                        content[key]['nerfacto']['metrics'][elem]['fps_std'] = data['results']['fps_std']
        if splatfacto:
            # Splatfacto
            if content[key]['splatfacto'].get('metrics') is None:
                content[key]['splatfacto']['metrics'] = {}
            if os.path.exists(os.path.join(path + '_splatfacto', key)):
                for elem in elems:
                    if os.path.exists(os.path.join(path + '_splatfacto', key, 'output_'+ str(elem) + '.json')):
                        with open(os.path.join(path + '_splatfacto', key, 'output_'+ str(elem) + '.json')) as file:
                            data = json.load(file)
                        if content[key]['splatfacto']['metrics'].get(elem) is None:
                            content[key]['splatfacto']['metrics'][elem] = {}
                        content[key]['splatfacto']['metrics'][elem]['psnr'] = data['results']['psnr']
                        content[key]['splatfacto']['metrics'][elem]['psnr_std'] = data['results']['psnr_std']
                        content[key]['splatfacto']['metrics'][elem]['ssim'] = data['results']['ssim']
                        content[key]['splatfacto']['metrics'][elem]['ssim_std'] = data['results']['ssim_std']
                        content[key]['splatfacto']['metrics'][elem]['lpips'] = data['results']['lpips']
                        content[key]['splatfacto']['metrics'][elem]['lpips_std'] = data['results']['lpips_std']
                        content[key]['splatfacto']['metrics'][elem]['num_rays_per_sec'] = data['results']['num_rays_per_sec']
                        content[key]['splatfacto']['metrics'][elem]['num_rays_per_sec_std'] = data['results']['num_rays_per_sec_std']
                        content[key]['splatfacto']['metrics'][elem]['fps'] = data['results']['fps']
                        content[key]['splatfacto']['metrics'][elem]['fps_std'] = data['results']['fps_std']

def add_laplacians(path, content):
    for k, key in enumerate(content.keys()):
        print(f'\n{k} feitos')
        diri = os.path.join(path, key, 'images')
        values = compute_laplacian(diri)
        content[key]['laplacians'] = values.tolist()

def add_ram(path, content, nerfacto=True, splatfacto=True):
    # colmap
    diri = os.path.join(path, 'colmap', 'RAM')
    for k, key in enumerate(content.keys()):
        with open(os.path.join(diri, key + '_ram.txt')) as file:
            df = pd.read_csv(file, sep='\s+')
        df = df.loc['Mem.:']
        total = np.array(df['total']).astype(np.float32)
        usada = np.array(df['usada']).astype(np.float32)
        livre = np.array(df['livre']).astype(np.float32)
        compart = np.array(df['compart.']).astype(np.float32)
        buff_cache = np.array(df['buff/cache']).astype(np.float32)
        disponivel = np.array(df['disponível']).astype(np.float32)
        used = (total - disponivel) / 1e6 # in Gb
        content[key]['colmap']['ram'] = used.tolist()
        print_progress_bar(k, len(content))

    print('\n')

    if nerfacto:
        # nerfacto
        diri = os.path.join(path, 'nerfacto', 'RAM')
        for k, key in enumerate(content.keys()):
            with open(os.path.join(diri, key + '_ram.txt')) as file:
                df = pd.read_csv(file, sep='\s+')
            df = df.loc['Mem.:']
            total = np.array(df['total']).astype(np.float32)
            usada = np.array(df['usada']).astype(np.float32)
            livre = np.array(df['livre']).astype(np.float32)
            compart = np.array(df['compart.']).astype(np.float32)
            buff_cache = np.array(df['buff/cache']).astype(np.float32)
            disponivel = np.array(df['disponível']).astype(np.float32)
            used = (total - disponivel) / 1e6 # in Gb
            content[key]['nerfacto']['ram'] = used.tolist()
            print_progress_bar(k, len(content))

    print('\n')

    if splatfacto:
        # splatfacto
        diri = os.path.join(path, 'splatfacto', 'RAM')
        for k, key in enumerate(content.keys()):
            with open(os.path.join(diri, key + '_ram.txt')) as file:
                df = pd.read_csv(file, sep='\s+')
            df = df.loc['Mem.:']
            total = np.array(df['total']).astype(np.float32)
            usada = np.array(df['usada']).astype(np.float32)
            livre = np.array(df['livre']).astype(np.float32)
            compart = np.array(df['compart.']).astype(np.float32)
            buff_cache = np.array(df['buff/cache']).astype(np.float32)
            disponivel = np.array(df['disponível']).astype(np.float32)
            used = (total - disponivel) / 1e6 # in Gb
            content[key]['splatfacto']['ram'] = used.tolist()
            print_progress_bar(k, len(content))

def add_gpu(path, content, nerfacto=True, splatfacto=True):
    # colmap
    diri = os.path.join(path, 'colmap', 'GPU')
    for k, key in enumerate(content.keys()):
        with open(os.path.join(diri, key + '_gpu.txt')) as file:
            df = pd.read_csv(file, sep='\s+', header=None)
        content[key]['colmap']['gpu'] = list(df[0])
        print_progress_bar(k, len(content))
    
    print('\n')

    if nerfacto:
        # nerfacto
        diri = os.path.join(path, 'nerfacto', 'GPU')
        for k, key in enumerate(content.keys()):
            with open(os.path.join(diri, key + '_gpu.txt')) as file:
                df = pd.read_csv(file, sep='\s+', header=None)
            content[key]['nerfacto']['gpu'] = list(df[0])
            print_progress_bar(k, len(content))

    print('\n')

    if splatfacto:
        # splatfacto
        diri = os.path.join(path, 'splatfacto', 'GPU')
        for k, key in enumerate(content.keys()):
            with open(os.path.join(diri, key + '_gpu.txt')) as file:
                df = pd.read_csv(file, sep='\s+', header=None)
            content[key]['splatfacto']['gpu'] = list(df[0])
            print_progress_bar(k, len(content))

def add_percentage_images(path, content, pilot_study=True):
    files = []
    for file in os.listdir(path):
        if os.path.isdir(os.path.join(path, file)) and not file.startswith('.'):
            files.append(file)
    if pilot_study:
        for k, file in enumerate(files):    
            vec = []
            for i in range(1,11):
                image_path = os.path.join(path, file, 'teste_' + str(i) + '/colmap/sparse/0/images.bin')
                if os.path.exists(image_path):
                    vec.append(len(read_images_binary(image_path)))
            print_progress_bar(k, len(files))
            vec = np.array(vec) / content[file]['number_images'] * 100
            content[file]['percentage_frames'] = vec.tolist()
            content[file]['percentage'] = sum(i for i in vec if i == 100) / 10

# Preprocessing and data saving

## Extract time

In [28]:
content = extract_time('../../new gaussians/tempo_gasto.txt', nerfacto=True, splatfacto=False)

## Extract number of frames

In [34]:
videos_path = '/media/user/0E94B37D94B365BD/Users/user/Documents/new_gaussians'
count_number_images(videos_path, content)

## Extract RAM

In [29]:
ram_path = '/home/user/Documentos/Python/Artigo/new gaussians'
add_ram(ram_path, content, nerfacto=True, splatfacto=False)

Progress: [█████████████████████████████████████             ] 75.00%



Progress: [█████████████████████████████████████             ] 75.00%

## Extract GPU

In [30]:
gpu_path = '/home/user/Documentos/Python/Artigo/new gaussians'
add_gpu(gpu_path, content, nerfacto=True, splatfacto=False)

Progress: [█████████████████████████████████████             ] 75.00%



Progress: [█████████████████████████████████████             ] 75.00%

## Extract percentage of right colmaps

In [31]:
path = '/media/user/0E94B37D94B365BD/Users/user/Documents/new_gaussians'
add_percentage_images(path, content, pilot_study=True)

## Extract laplacians

In [15]:
laplacians_path = '/media/user/0E94B37D94B365BD/Users/user/Documents/new_gaussians'
add_laplacians(laplacians_path, content)


0 feitos
Progress: [█████████████████████████████████████████████████ ] 98.68%
1 feitos
Progress: [██████████████████████████████████████████████    ] 92.59%
2 feitos
Progress: [███████████████████████████████████████████       ] 86.46%
3 feitos
Progress: [██████████████████████████████████████████████    ] 93.75%

## Extract evaluations

In [32]:
evaluations_path = '/home/user/Documentos/Python/Artigo/new gaussians/Evaluations'
add_evaluations(evaluations_path, content, nerfacto=True, splatfacto=False)

## Save data in a .json file

In [18]:
## Save json
json_object = json.dumps(content, indent = 2)
with open('../../new gaussians/new_gaussians.json', 'w') as file:
    file.write(json_object)
    file.close()

In [36]:
content['bloco1-volta-completa']['splatfacto'].keys()

dict_keys(['time', 'ram', 'gpu', 'metrics'])